In [1]:
import numpy as np
import os
import pickle
## for the original grover
def grover_unitary(nqubit,n):
    #famous -I+2A gate 
    inv = -np.eye(1<<nqubit,dtype=np.complex128)+2*np.ones((1<<nqubit,1<<nqubit),dtype=np.complex128)/(1<<nqubit)
    return [inv for i in range(n)]

def oracle(state,index):
    state[index] *= -1
    return state


def apply_circuit(state,unitary_list,index):
    for i in range(len(unitary_list)-1):
        state = unitary_list[i]@state
        state = oracle(state,index)
    state = unitary_list[-1]@state
    return state

def apply_circuit_all(state,unitary_list):
    success_rate = []
    for i in range(state.shape[0]):
        tmp = np.copy(state)
        tmp = apply_circuit(tmp,unitary_list,i)
        success_rate.append(np.abs(tmp[i])**2)
    return success_rate

## Success rate of search algorithm learned by VarQAA 

In [2]:
folder = "data/"
files=os.listdir(folder)
for file in files:
    if not os.path.isdir(file):
        with open(folder+file,"rb") as f:
            data =pickle.load(f)
        if data["use_fractional"]:
            continue
        else:
            num_qubit = data["num_qubit"]
            num_query = data["num_query"]
            unitary_list = data["unitary_list"]
            state = np.zeros(2**num_qubit,dtype=np.complex128)
            state[0]=1
            success_rate = apply_circuit_all(state,unitary_list)
            print("%d-qubit\t%d-query\tsuccess rate:%f"%(num_qubit,num_query,min(success_rate)))

5-qubit	3-query	success rate:0.896929
4-qubit	2-query	success rate:0.908428
4-qubit	3-query	success rate:1.000000
3-qubit	2-query	success rate:1.000000
3-qubit	1-query	success rate:0.781250
5-qubit	4-query	success rate:1.000000


## Fractional Grover algorithm

In [3]:
import numpy as np
import os
import pickle
## for the original grover
def grover_unitary(nqubit,n):
    #famous -I+2A gate 
    inv = -np.eye(1<<nqubit,dtype=np.complex128)+2*np.ones((1<<nqubit,1<<nqubit),dtype=np.complex128)/(1<<nqubit)
    return [inv for i in range(n)]

def frac_oracle(state,index,alpha):
    state[index] *= np.exp(-1j*np.pi*alpha)
    return state

def apply_circuit(state,unitary_list,alpha,index):
    for i in range(len(unitary_list)-1):
        state = unitary_list[i]@state
        state = frac_oracle(state,index,alpha[i])
    state = unitary_list[-1]@state
    return state

def apply_circuit_all(state,unitary_list,alpha):
    success_rate = []
    for i in range(state.shape[0]):
        tmp = np.copy(state)
        tmp = apply_circuit(tmp,unitary_list,alpha,i)
        success_rate.append(np.abs(tmp[i])**2)
    return success_rate

In [4]:
folder = "data/"
files=os.listdir(folder)
for file in files:
    if not os.path.isdir(file):
        with open(folder+file,"rb") as f:
            data =pickle.load(f)
        if data["use_fractional"]:
            num_qubit = data["num_qubit"]
            num_query = data["num_query"]
            unitary_list = data["unitary_list"]
            alpha = data["alpha"]
            state = np.zeros(2**num_qubit,dtype=np.complex128)
            state[0]=1
            success_rate = apply_circuit_all(state,unitary_list,alpha)
            print("%d-qubit\t%d-query\tsuccess rate:%f"%(num_qubit,num_query,min(success_rate)))
            print("alpha:",alpha)

4-qubit	3-query	success rate:1.000000
alpha: [0.9996027397671272, 0.9791097786250649, 0.9994421871467769]
5-qubit	4-query	success rate:1.000000
alpha: [0.9991000793247986, 0.9984690986935828, 0.9952702691214556, 0.9940184960156646]
3-qubit	2-query	success rate:1.000000
alpha: [0.5930521057667743, 0.9366006791817347]
